In [114]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

import sys
sys.path.insert(1, '../python')

from cb_preprocess import load_data, preprocess, tokenize, label_to_numeric
from distance_measures.multiset_distances import MultisetDistances
from collections import Counter
from random import shuffle
from nltk.util import ngrams
from nltk.translate import bleu_score

from tqdm.notebook import trange
from sklearn.model_selection import KFold

In [81]:
df = pd.read_csv("../../data/wang_preprocessed.csv", index_col=0)
df = df[df.lang == "en"]
df["token_list"] = ((df.tokens.str.replace("[^a-zA-Z0-9]", " "))
    .str.split("\s+"))
df = df.dropna(subset = ["token_list"]).reset_index(drop = True)
df

C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\2321408713.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["token_list"] = ((df.tokens.str.replace("[^a-zA-Z0-9]", " "))


,tweet_text,label,y,tokens,lang,token_list
0,Here at home. Neighbors pick on my family and ...,age,0,here at home . neighbors pick on my family and...,en,"[here, at, home, neighbors, pick, on, my, fami..."
1,Being bullied at school: High-achieving boys u...,age,0,being bullied at school : high - achieving boy...,en,"[being, bullied, at, school, high, achieving, ..."
2,There was a girl in my class in 6th grade who ...,age,0,there was a girl in my class in 6th grade who ...,en,"[there, was, a, girl, in, my, class, in, 6th, ..."
3,He’s probably a white gay kid from some suburb...,age,0,he ’ s probably a white gay kid from some subu...,en,"[he, s, probably, a, white, gay, kid, from, so..."
4,You are pushed ti resorting. Treating thr bull...,age,0,you are pushed ti resorting . treating thr bul...,en,"[you, are, pushed, ti, resorting, treating, th..."
...,...,...,...,...,...,...
46411,Can you imagine if Christians came together li...,religion,5,can you imagine if christians came together li...,en,"[can, you, imagine, if, christians, came, toge..."
46412,So how to support justice from the initial pro...,religion,5,so how to support justice from the initial pro...,en,"[so, how, to, support, justice, from, the, ini..."
46413,$RESERVED$ $MENTION$: If you harbour any doubt...,religion,5,$RESERVED$ $MENTION$ : if you harb ##our any d...,en,"[, RESERVED, MENTION, if, you, harb, our, any,..."
46414,"$MENTION$ $MENTION$ One thing about Muslims, t...",religion,5,"$MENTION$ $MENTION$ one thing about muslims , ...",en,"[, MENTION, MENTION, one, thing, about, muslim..."


In [117]:
## self-scored metrics
corpus = df.token_list.to_list()

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for k,  (train, test) in enumerate(k_fold.split(corpus)):
    print(f"Fold {k}")
    print(len(train))
    print(len(test))

    msd = MultisetDistances(references=np.array(corpus)[train], min_n=3, max_n=5)
    msj_distance = msd.get_canberra_score(sentences=np.array(corpus)[test])

    print(msj_distance)

Fold 0
37132
9284


C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\3860865462.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msd = MultisetDistances(references=np.array(corpus)[train], min_n=3, max_n=5)
C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\3860865462.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msj_distance = msd.get_canberra_score(sentences=np.array(corpus)[test])


{3: 151528.33169525987, 4: 238638.05489574914, 5: 315112.96478306124}
Fold 1
37133
9283
{3: 151649.64182165457, 4: 238782.50143318455, 5: 315266.9558190299}
Fold 2
37133
9283
{3: 151652.13951792673, 4: 238786.57750275676, 5: 315266.0576011363}
Fold 3
37133
9283
{3: 151303.4830489929, 4: 238386.92541865236, 5: 314856.0252331899}
Fold 4
37133
9283
{3: 151831.34408472423, 4: 238984.50850263773, 5: 315454.8130185417}


In [118]:
for label in df.label.unique():
    print(label + "\n------------------------------------------------", "\n")

    subset = df[df.label == label].token_list.to_list()
    for k,  (train, test) in enumerate(k_fold.split(subset)):
        print(f"Fold {k}")

        msd = MultisetDistances(references=np.array(subset)[train], min_n=3, max_n=5)
        msj_distance = msd.get_canberra_score(sentences=np.array(subset)[test])

        print(msj_distance)

age
------------------------------------------------ 

Fold 0


C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\4158265818.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msd = MultisetDistances(references=np.array(subset)[train], min_n=3, max_n=5)
C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\4158265818.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msj_distance = msd.get_canberra_score(sentences=np.array(subset)[test])


{3: 55547.86661749473, 4: 78281.11961876924, 5: 96879.62006380392}
Fold 1
{3: 55568.76384844144, 4: 78305.40318342981, 5: 96905.8786425161}
Fold 2
{3: 55543.38508983946, 4: 78270.04426685032, 5: 96864.58134949439}
Fold 3
{3: 55495.83082060567, 4: 78220.6306740972, 5: 96818.62308366131}
Fold 4
{3: 55606.20660853682, 4: 78341.25121833115, 5: 96934.47724978738}
ethnicity
------------------------------------------------ 

Fold 0
{3: 47948.480584768244, 4: 65418.56270798026, 5: 78957.33810605631}
Fold 1
{3: 48024.88971105231, 4: 65496.91319821616, 5: 79034.90702085193}
Fold 2
{3: 48037.44539364074, 4: 65504.78304845601, 5: 79037.18992176288}
Fold 3
{3: 48004.42853036234, 4: 65475.009566463756, 5: 79009.98835136881}
Fold 4
{3: 48116.006376293495, 4: 65595.73978002318, 5: 79131.52245375847}
gender
------------------------------------------------ 

Fold 0
{3: 46571.01623298052, 4: 63450.62019069445, 5: 76567.34000537303}
Fold 1
{3: 46616.637714213575, 4: 63497.64836704607, 5: 76614.24545382173

In [120]:
for label in df.label.unique():
    print("\n" + label + " Similarity Matrix\n------------------------------------------------", "\n")
    msd = MultisetDistances(references=np.array(df[df.label == label].token_list.to_list()), min_n=3, max_n=5)

    for comparison_label in df.label.unique(): 
        msj_distance = msd.get_jaccard_score(sentences=np.array(df[df.label == comparison_label].token_list.to_list()))
        print(f"{label} vs {comparison_label}: {msj_distance}")


age Similarity Matrix
------------------------------------------------ 



C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\3174125392.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msd = MultisetDistances(references=np.array(df[df.label == label].token_list.to_list()), min_n=3, max_n=5)
C:\Users\Ayush\AppData\Local\Temp\ipykernel_92912\3174125392.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  msj_distance = msd.get_jaccard_score(sentences=np.array(df[df.label == comparison_label].token_list.to_list()))


age vs age: {3: 1.0, 4: 1.0, 5: 1.0}
age vs ethnicity: {3: 0.13887941278644164, 4: 0.06679257854139722, 5: 0.030290716860450838}
age vs gender: {3: 0.14502734715966326, 4: 0.06996326158120537, 5: 0.03151066110010643}
age vs notcb: {3: 0.11302129230470814, 4: 0.05582762455751118, 5: 0.025951137629197884}
age vs other: {3: 0.12689593075150177, 4: 0.06392381921428818, 5: 0.030344426762078772}
age vs religion: {3: 0.1360006484570393, 4: 0.0633674383278422, 5: 0.02753741557424664}

ethnicity Similarity Matrix
------------------------------------------------ 

ethnicity vs age: {3: 0.13887941278645996, 4: 0.06679257854140833, 5: 0.03029071686045657}
ethnicity vs ethnicity: {3: 1.0, 4: 1.0, 5: 1.0}
ethnicity vs gender: {3: 0.1595358320746188, 4: 0.07826843782363867, 5: 0.037034363416590235}
ethnicity vs notcb: {3: 0.13326542542048475, 4: 0.06621808823256906, 5: 0.03187850762518544}
ethnicity vs other: {3: 0.14661623820721303, 4: 0.07401877186707038, 5: 0.03640609785756408}
ethnicity vs religi